In [ ]:
# ---
# Run SQL with Windows Authentication in a Jupyter/Databricks Notebook
# ---
# Prerequisites:
#   pip install pyodbc pandas
#   Install Microsoft ODBC Driver 17/18 for SQL Server
# ---

import pyodbc
import pandas as pd

# === PARAMETERS ===
server = r"YOURSERVER\SQLEXPRESS"     # e.g. 'MYSERVER\\SQLEXPRESS' (double backslash)
database = "YourDatabaseName"
driver = "ODBC Driver 18 for SQL Server"  # or "ODBC Driver 17 for SQL Server"
load_task_id = 13449
company_issuer_id = 2204
out_csv = "mapping_results.csv"

# Optional flags
encrypt = False
trust_server_certificate = True

# === CONNECTION ===
conn_parts = [
    f"DRIVER={{{driver}}}",
    f"SERVER={server}",
    f"DATABASE={database}",
    "Trusted_Connection=yes",
    f"Encrypt={'yes' if encrypt else 'no'}",
    f"TrustServerCertificate={'yes' if trust_server_certificate else 'no'}"
]
conn_str = ";".join(conn_parts)

print("Connecting with connection string:")
print(conn_str)

conn = pyodbc.connect(conn_str)
print("Connected successfully ✅")

# === SQL QUERY ===
sql = """
DECLARE @LoadTaskId INT = ?;
DECLARE @CompanyIssuerId INT = ?;

SET NOCOUNT ON;

SELECT
    s.PolicyNumber,
    cpp.ProductName
FROM dbo.STGPlanMappingFactors s (NOLOCK)
OUTER APPLY (
    SELECT TOP 1 p.*
    FROM dbo.PlanMappingIssuerDetail p (NOLOCK)
    LEFT JOIN dbo.PlanMappingIssuerDetailState ps (NOLOCK)
        ON p.PlanMappingIssuerDetailId = ps.PlanMappingIssuerDetailId
    WHERE
        (p.IssuerPlanName = s.PlanName OR p.IssuerPlanName IS NULL OR p.IssuerPlanName = '')
        AND (p.IssuerPlanId = s.PlanId OR p.IssuerPlanId IS NULL OR p.IssuerPlanId = '')
        AND (p.[Year] = s.[Year] OR p.[Year] IS NULL)
        AND (p.PercentRate = s.RatePercent OR p.PercentRate IS NULL)
        AND (p.CurrencyRate = s.RateDollar OR p.CurrencyRate IS NULL)
        AND (ps.StateCode = s.StateCode OR ps.StateCode IS NULL)
    ORDER BY
        CASE WHEN p.IssuerPlanName = s.PlanName THEN 1 ELSE 0 END +
        CASE WHEN p.IssuerPlanId   = s.PlanId   THEN 1 ELSE 0 END +
        CASE WHEN p.PercentRate    = s.RatePercent THEN 1 ELSE 0 END +
        CASE WHEN p.[Year]         = s.[Year]   THEN 1 ELSE 0 END +
        CASE WHEN p.CurrencyRate   = s.RateDollar THEN 1 ELSE 0 END +
        CASE WHEN ps.StateCode     = s.StateCode THEN 1 ELSE 0 END
        DESC
) p
JOIN dbo.CompanyProduct cp WITH (NOLOCK)
    ON cp.Deleted = 0
   AND cp.CompanyIssuerId = @CompanyIssuerId
JOIN dbo.CompanyProductPlan cpp WITH (NOLOCK)
    ON cpp.Deleted = 0
   AND cpp.CompanyProductId = cp.CompanyProductId
   AND p.CompanyProductPlanId = cpp.CompanyProductPlanId
WHERE s.LoadTaskId = @LoadTaskId;
"""

# === EXECUTE AND FETCH ===
df = pd.read_sql(sql, conn, params=[load_task_id, company_issuer_id])
print(f"✅ Query executed successfully. Rows returned: {len(df)}")

# === PREVIEW & EXPORT ===
display(df.head(10))  # Jupyter or Databricks display
df.to_csv(out_csv, index=False)
print(f"Results saved to: {out_csv}")

conn.close()
print("Connection closed.")
